In [34]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set()

from sklearn.model_selection import cross_val_score

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor, StackingRegressor

import pickle
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import os
import joblib

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

x_train = pd.read_excel('data/final_train.xlsx')
y_train = pd.read_excel('data/y_train.xlsx')
x_test = pd.read_excel('data/final_test.xlsx')

x_train.drop(columns = 'ID', inplace= True)
x_test.drop(columns = 'ID', inplace= True)
y_train.drop(columns = 'ID', inplace= True)

print(x_train.shape, y_train.shape)
print(x_test.shape)

np.set_printoptions(formatter={'float_kind':'{:f}'.format})

(7000, 171) (7000, 1)
(3000, 171)


# Train test split
for purpose of this notebook, we will just use training set as original data to be split into train and test. Once we perform data cleaning + feature engineering for full dataset George sent we can just replace that here. 

In [36]:
X_train, X_test, y_train, y_test = train_test_split(x_train.fillna(0), y_train, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(5600, 171) (1400, 171)
(5600, 1) (1400, 1)


# Process of Scoring

Below is our method of how we will conduct this experiment.  
1. Run Grid Search CV for each model to find optimal parameters based on training set CV averages
    * some models will require pipelining with standard scaler etc
    * add final model to master dictionary
2. Pickle and save files to models directory
3. Loop through master model dictionary to get eval metrics for train and test 
4. Finalise table of results

# 1. Finding Optimised Models 

In [25]:
master_models = {}

In [26]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# add to dict 
master_models['Linear Regression'] = lin_reg

In [28]:
pipe_lasso = make_pipeline(StandardScaler(),
                           Lasso(random_state= 42))

param_lasso = [{'lasso__alpha':[1, 1.5, 2, 2.5, 3, 4, 4.5, 8, 10, 20, 50]}]

gs_lasso = GridSearchCV(estimator=pipe_lasso,
                        param_grid=param_lasso,
                        cv=5,
                        scoring = 'neg_mean_absolute_error',
                        n_jobs=-1
                        )
gs_lasso.fit(X_train, y_train)
print(gs_lasso.best_estimator_)
print(gs_lasso.best_score_)
gs_lasso_best_model = gs_lasso.best_estimator_

# add to dict 
master_models['Lasso'] = gs_lasso_best_model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=20, random_state=42))])
-118.81177014199424


In [32]:
pipe_ridge = make_pipeline(StandardScaler(), 
                           Ridge(random_state= 42))

param_ridge= [{'ridge__alpha':[1, 1.5, 2, 2.5, 3, 4, 4.5, 8, 10, 20, 50]}]

gs_ridge = GridSearchCV(estimator=pipe_ridge,
                        param_grid=param_ridge,
                        cv=5,
                        scoring = 'neg_mean_absolute_error',
                        n_jobs=-1
                        )
gs_ridge.fit(X_train, y_train)
print(gs_ridge.best_estimator_)
print(gs_ridge.best_score_)
gs_ridge_best_model = gs_ridge.best_estimator_

# add to dict 
master_models['Ridge'] = gs_ridge_best_model


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=50, random_state=42))])
-125.4870544471482


In [37]:
pipe_elastic = make_pipeline(StandardScaler(), 
                             ElasticNet(random_state= 42))
param_elastic = [{'elasticnet__l1_ratio':[0.5, 0.6, 0.7],
                  'elasticnet__alpha':[0.0001, 0.001, 0.01]}]
gs_elastic = GridSearchCV(estimator=pipe_elastic,
                          param_grid=param_elastic,
                          cv=5,
                          scoring = 'neg_mean_absolute_error',
                          n_jobs=-1)

gs_elastic.fit(X_train, y_train)
print(gs_elastic.best_estimator_)
print(gs_elastic.best_score_)
gs_elastic_best_model = gs_elastic.best_estimator_

# add to dict 
master_models['Elastic Net'] = gs_elastic_best_model

/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.522e+07, tolerance: 5.130e+04
  model = cd_fast.enet_coordinate_descent(
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.072e+08, tolerance: 5.595e+04
  model = cd_fast.enet_coordinate_descent(
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check t

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('elasticnet', ElasticNet(alpha=0.01, random_state=42))])
-125.93340510067064


In [40]:
from sklearn.neighbors import KNeighborsRegressor

pipe_knn = make_pipeline(StandardScaler(),
                         KNeighborsRegressor())

param_knn = {'kneighborsregressor__n_neighbors' : range(3, 20, 2)}

gs_knn = GridSearchCV(estimator=pipe_knn,
                      param_grid=param_knn,
                      cv=5,
                      scoring = 'neg_mean_absolute_error',
                      n_jobs=-1)
gs_knn.fit(X_train, y_train)
print(gs_knn.best_estimator_)
print(gs_knn.best_score_)
gs_knn_best_model = gs_knn.best_estimator_

# add to dict 
master_models['KNN Regressor'] = gs_knn_best_model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=7))])
-110.0632142857143


In [41]:
pipe_svr = make_pipeline(StandardScaler(),
                         SVR(kernel='rbf'))

param_svr = {'svr__C' : [1000,1500,2000,2500],
             'svr__gamma' : [1,0.1,0.01, 0.001,0.0001]}

gs_svr = GridSearchCV(estimator=pipe_svr,
                      param_grid=param_svr,
                      cv=5,
                      scoring = 'neg_mean_absolute_error',
                      n_jobs=-1)
gs_svr.fit(X_train, y_train)
print(gs_svr.best_estimator_)
print(gs_svr.best_score_)
gs_svr_best_model = gs_svr.best_estimator_

# add to dict 
master_models['Support Vector Regressor'] = gs_svr_best_model

/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/utils/validation.py:114

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(C=2000, gamma=0.001))])
-86.55658075263105


In [46]:
dt= DecisionTreeRegressor(random_state=42)

param_dt = {'max_features': ['auto', 'sqrt', 'log2'],
            'min_samples_split': [2,3,4,5,6,7,8,9,10],
            'min_samples_leaf':[1,2,3,4,5, 8, 10],
            'max_depth':[2,4,6,8],
            'random_state':[42]
            }

gs_dt = GridSearchCV(dt, 
                     param_grid=param_dt, 
                     cv=5,
                     scoring = 'neg_mean_absolute_error', 
                     n_jobs=-1)
gs_dt.fit(X_train, y_train)
print(gs_dt.best_estimator_)
print(gs_dt.best_score_)
gs_dt_best_model = gs_dt.best_estimator_

# add to dict 
master_models['Decision Tree'] = gs_dt_best_model

/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1

DecisionTreeRegressor(max_depth=6, max_features='auto', min_samples_leaf=4,
                      random_state=42)
-108.64030490378988


In [49]:
import warnings
warnings.filterwarnings("ignore")
rf= RandomForestRegressor(random_state=42)

param_rf = [{'n_estimators': [100,400,800,1200], 
               'max_features': [10,15,20], 
               'max_depth': [5,6,7]}
]

gs_rf = GridSearchCV(rf,  
                     param_grid=param_rf, 
                     cv=5,
                     scoring = 'neg_mean_absolute_error', 
                     n_jobs=-1)
gs_rf.fit(X_train, y_train)
print(gs_rf.best_estimator_)
print(gs_rf.best_score_)
gs_rf_best_model = gs_rf.best_estimator_

# add to dict 
master_models['Random Forest'] = gs_rf_best_model

/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/domnasrabadi/opt/anaconda3/

RandomForestRegressor(max_depth=7, max_features=20, n_estimators=1200,
                      random_state=42)
-97.1869602565009


In [51]:
xgb = XGBRegressor()

param_xgb = [{'n_estimators': [50,100,300, 500],
              'min_child_weight': [1,10,100],
              'learning_rate': [0.01,0.1,1],
              'max_depth': [2,3,4,5,8,12]}]

gs_xgb = GridSearchCV(xgb,  
                     param_grid=param_xgb, 
                     cv=5,
                     scoring = 'neg_mean_absolute_error', 
                     n_jobs=-1)
gs_xgb.fit(X_train, y_train)
print(gs_xgb.best_estimator_)
print(gs_xgb.best_score_)
gs_xgb_best_model = gs_xgb.best_estimator_

# add to dict 
master_models['XGBoost'] = gs_xgb_best_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)
-86.10284660546907


In [52]:
gb= GradientBoostingRegressor()

param_gb = [{'learning_rate': [0.01,0.1,1],
             'n_estimators' : [10,50,100],
             'max_depth'    : [1,2,3,4,8],
             'alpha' : [0.1,0.5,0.9]
             }]
gs_gb = GridSearchCV(gb,  
                     param_grid=param_gb, 
                     cv=4,
                     scoring = 'neg_mean_absolute_error', 
                     n_jobs=-1)
gs_gb.fit(X_train, y_train)
print(gs_gb.best_estimator_)
print(gs_gb.best_score_)
gs_gb_best_model = gs_gb.best_estimator_

# add to dict 
master_models['Gradient Boosting'] = gs_gb_best_model

/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/domnasrabadi/opt/anaconda3/envs/S1_2023/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWar

GradientBoostingRegressor(max_depth=4)
-88.70505111658643


### Below keeps crashing

In [1]:

# lgbm = LGBMRegressor()

# param_lgbm = [{
#     # 'objective': ['root_mean_squared_error'],
#     # 'num_iterations': [1000, 2000, 3000],
#     'learning_rate':[0.01, 0.03, 0.05],
#     'max_depth' :[1, 3, 5, 8],
#     'feature_fraction': [0.6, 0.8]}]

# gs_lgbm = GridSearchCV(lgbm, 
#                         param_grid=param_lgbm, 
#                         cv=4,
#                         scoring = 'neg_mean_absolute_error', 
#                         n_jobs=-1)

# gs_lgbm.fit(X_train, y_train)
# print(gs_lgbm.best_estimator_)
# print(gs_lgbm.best_score_)
# gs_lgbm_best_model = gs_lgbm.best_estimator_

# # add to dict 
# master_models['Light GBM'] = gs_lgbm_best_model

In [ ]:
mlpregr = MLPRegressor(random_state=1, max_iter=500)
pipe_mlp = make_pipeline(StandardScaler(),mlpregr)
param_mlp = {
    'mlpregressor__hidden_layer_sizes': [(10,30,10),(20,)],
    #'activation': ['tanh', 'relu'],
    #'solver': ['sgd', 'adam'],
    'mlpregressor__alpha': [0.0001, 0.05],
    'mlpregressor__learning_rate': ['constant','adaptive'],
}
gs_mlp = GridSearchCV(estimator=pipe_mlp,
                      param_grid=param_mlp,
                      cv=5,
                      n_jobs=-1)
gs_mlp.fit(X_train, y_train)
print(gs_mlp.best_estimator_)
print(gs_mlp.best_score_)
# gs_mlp_best_model = gs_mlp.best_estimator_

# # add to dict 
# master_models['MLP'] = gs_mlp_best_model